In [1]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 05: Task Pattern (A2A)

This notebook demonstrates the **Task Communication Pattern** using the **Agent-to-Agent (A2A) protocol** - an open standard that enables one agent to delegate work to another agent by creating and managing tasks.

## Understanding the Task Pattern

The **Task Pattern** is a communication approach where:
- One agent (the **delegator**) creates a task for another agent (the **worker**)
- The worker agent processes the task independently
- The worker updates the task with results/artifacts when complete
- Communication happens asynchronously via task status and artifacts

## A2A Protocol: An Implementation of the Task Pattern

The **A2A (Agent-to-Agent) protocol**, introduced by Google, is a standardized HTTP-based implementation of the Task Pattern. It defines:
- How to create tasks with specific goals
- How agents communicate task status
- How to exchange artifacts (results/data)
- A standard message format for interoperability

## Task Pattern Architecture

```mermaid
sequenceDiagram
    participant User as 👤 User
    participant Agent1 as 🤖 Agent 1<br/>(Financial Assistant)<br/>Delegator
    participant A2A as 📡 A2A Protocol<br/>(HTTP/Task API)
    participant Agent2 as 📊 Agent 2<br/>(Stock Analysis)<br/>Worker
    
    User->>Agent1: Request stock analysis for AAPL
    
    Note over Agent1: Decides to delegate<br/>to specialist
    
    Agent1->>A2A: POST /tasks<br/>Create Task<br/>{goal: "Analyze AAPL stock"}
    A2A->>Agent2: Forward task creation
    
    Note over Agent2: Task Created<br/>Status: pending
    
    Agent2-->>A2A: Task Status: working
    A2A-->>Agent1: Task accepted
    
    Note over Agent2: Processes task:<br/>- Web search<br/>- Data analysis<br/>- Generate report
    
    Agent2->>Agent2: Execute analysis
    
    Agent2->>A2A: PUT /tasks/{id}<br/>Update Task<br/>Status: complete<br/>Artifacts: [StockReport]
    
    A2A->>Agent1: Task complete notification<br/>Artifacts available
    
    Agent1->>A2A: GET /tasks/{id}/artifacts
    A2A->>Agent1: Return StockReport artifact
    
    Agent1->>User: Present analysis results
    
    Note over Agent1,Agent2: Task Pattern Benefits:<br/>✓ Asynchronous execution<br/>✓ Independent agents<br/>✓ Standardized protocol<br/>✓ Scalable architecture
    
    style User fill:#e1f5ff,stroke:#01579b,stroke-width:2px
    style Agent1 fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style A2A fill:#ffebee,stroke:#c62828,stroke-width:3px
    style Agent2 fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
```

**Key Components:**
1. **Task Creation**: Agent 1 creates a task with a specific goal
2. **Task Delegation**: A2A protocol delivers the task to Agent 2
3. **Task Processing**: Agent 2 works on the task independently
4. **Task Updates**: Agent 2 updates task status during processing
5. **Artifact Exchange**: Agent 2 attaches results as artifacts to the task
6. **Task Completion**: Agent 1 retrieves the completed task with artifacts

**⚠️ IMPORTANT**: Before running this notebook, start the A2A stock analysis server in a separate terminal:

```bash
uvicorn lecture05.stock_a2a_server:app --host 0.0.0.0 --port 8001
```

**Reference**: [Pydantic AI A2A Documentation](https://ai.pydantic.dev/a2a/)


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging
from lecture05.agent import a2a_stock_analysis_tool  # Pre-built A2A tool

# Initialize logging
setup_logging()

In [ ]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="task-pattern-a2a-lecture05",
    start_message="🚀 Lecture 05 - Task Pattern (A2A) Notebook Started",
)

In [ ]:
# Create financial assistant with Task Pattern (A2A) delegation capability
# The a2a_stock_analysis_tool implements the Task Pattern:
# 1. Creates a task for the stock analysis agent
# 2. The worker agent processes the task independently
# 3. Results are returned as task artifacts via A2A protocol
a2a_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, a2a_stock_analysis_tool],
    system_prompt="""
You are a knowledgeable financial assistant using the Task Pattern for delegation.

When answering questions:
1. Use web_search for general financial information
2. Use request_stock_analysis_a2a to delegate stock analysis tasks
   - This creates a TASK for the specialist stock analysis agent
   - The task is sent via A2A protocol (HTTP) to a worker agent
   - The worker processes the task and returns results as artifacts
   - The worker agent runs as a separate A2A server at localhost:8001
3. Explain to users when you're delegating tasks to specialist agents

This demonstrates the Task Pattern where you delegate specialized work to other agents.

Keep your responses informative and concise.
""",
)

In [ ]:
# Ask a general financial question (uses web_search)
question = "What is the current price of Bitcoin?"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Ask for detailed stock analysis (uses A2A protocol)
# This will:
# 1. Send HTTP request to A2A server at localhost:8001
# 2. Use A2A protocol message format
# 3. Receive StockReport from A2A server
question = "Can you give me a detailed analysis of Apple stock (AAPL)?"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Try another stock analysis via Task Pattern delegation
# Notice how the delegator agent creates tasks and the worker agent
# independently processes them, returning results as artifacts
question = "Give me a comprehensive report on Tesla (TSLA)"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

## Comparison: Agent-as-Tool vs. A2A Protocol

| Aspect | Lecture 03 (Agent-as-Tool) | Lecture 05 (A2A Protocol) |
|--------|---------------------------|---------------------------|
| Pattern | Direct code integration | HTTP protocol communication |
| Communication | In-process function call | HTTP requests to A2A server |
| Protocol | None | Google's A2A standard |
| Agent Location | Same process | Separate service/server |
| Interoperability | Framework-specific | Cross-framework/vendor |
| Deployment | Single application | Distributed services |
| Scalability | Limited to process | Horizontally scalable |
